# 🚀 EGT Prototype — Embedded GAN-in-Transformer (v0.3)

This notebook demonstrates a **self-teaching Transformer** architecture where a **GAN** is embedded *inside* the Transformer blocks themselves.

Instead of standard training, the model alternates between two modes:
1. **Wake** — Generates responses to prompts, scores itself with internal critic networks, and stores experiences in a replay buffer.
2. **Sleep** — Replays those experiences through a *GAN-like* adversarial loop *inside* the Transformer, refining both its responses and its ability to refine its own responses.

### 🔹 Key Innovations
- **Embedded GAN**: Generator & Discriminators (critics) coexist inside Transformer layers.
- **LoRA Adaptation**: Lightweight low-rank adapters for efficient continual updates.
- **EMA Teacher Model**: Stabilizes learning over “lifetimes.”
- **Replay Buffer Memory**: Stores and reuses lived experiences.
- **Artificial Sleep Cycle**: The model “dreams” by training on its own outputs, improving its self-critique skills.

### 🧠 Conceptual Flow
Wake: Prompt → Generate → Self-Score → Store
Sleep: Replay → Discriminator vs. Generator → Teacher Alignment → Update


### 💡 Why It’s Interesting
This is **not** a typical “Transformer + GAN” pipeline.
- The **GAN lives *inside* the Transformer module** itself.
- The model **learns how to teach itself** better over time.
- Inspired by biological learning patterns (wake/sleep cycles).

### 📜 Included Features
- Tiny **SentencePiece-style BPE tokenizer** → readable outputs.
- Small demo corpus for quick tests.
- Configurable “dream” length and critic placement.
- Fully runnable on CPU for proof-of-concept.

---

⚠️ **Note:** This is a research prototype, not a production-ready model.
Scaling it up could lead to unexpected emergent behaviors — handle responsibly.

In [ ]:
from __future__ import annotations
import math
import random
from dataclasses import dataclass
from typing import List, Tuple, Dict

import torch
import torch.nn as nn
import torch.nn.functional as F

# =========================
# Tiny BPE Tokenizer (SentencePiece-style)
# =========================
class TinyBPETokenizer:
    """Minimal BPE with ▁ as word boundary.
    Trains merges on a small corpus; encodes by greedy longest-match; decodes back.
    """
    def __init__(self, texts: List[str], vocab_size: int = 200):
        self.pad = '<pad>'; self.bos = '<bos>'; self.eos = '<eos>'
        self.special = [self.pad, self.bos, self.eos]
        # Pre-tokenize words with leading boundary
        words = []
        for t in texts:
            for w in t.strip().split():
                if not w:
                    continue
                words.append('▁' + w)
        # Build initial vocab from characters
        vocab = set(self.special)
        # represent words as tuple of chars
        corpus = []
        for w in words:
            syms = tuple(list(w))
            corpus.append(syms)
            vocab.update(syms)
        def count_pairs(corpus_list):
            counts: Dict[Tuple[str,str], int] = {}
            for syms in corpus_list:
                for i in range(len(syms)-1):
                    pair = (syms[i], syms[i+1])
                    counts[pair] = counts.get(pair, 0) + 1
            return counts
        merges_list: List[Tuple[str,str]] = []
        # Train merges until vocab size target
        while True:
            tokens_so_far = {s for syms in corpus for s in syms} | set(['▁']) | set(self.special)
            if len(tokens_so_far) + len(merges_list) >= vocab_size:
                break
            pair_counts = count_pairs(corpus)
            if not pair_counts:
                break
            best_pair = max(pair_counts.items(), key=lambda x: x[1])[0]
            merges_list.append(best_pair)
            a,b = best_pair
            new_corpus = []
            for syms in corpus:
                i = 0; merged = []
                while i < len(syms):
                    if i < len(syms)-1 and syms[i] == a and syms[i+1] == b:
                        merged.append(a+b); i += 2
                    else:
                        merged.append(syms[i]); i += 1
                new_corpus.append(tuple(merged))
            corpus = new_corpus
        # Build final vocab mapping
        tokens = self.special + sorted({s for syms in corpus for s in syms} | set(['▁']))
        for a,b in merges_list:
            tok = a+b
            if tok not in tokens:
                tokens.append(tok)
        self.itos = tokens
        self.stoi = {t:i for i,t in enumerate(self.itos)}
        self.pad_id = self.stoi[self.pad]
        self.bos_id = self.stoi[self.bos]
        self.eos_id = self.stoi[self.eos]
        # Rank of merges for greedy encode
        self.merges = {(a+b):(i+1) for i,(a,b) in enumerate(merges_list)}

    @property
    def vocab_size(self):
        return len(self.itos)

    def encode(self, s: str, add_bos=True, add_eos=True) -> List[int]:
        pieces: List[str] = []
        for w in s.strip().split():
            if not w: continue
            syms = list('▁' + w)
            # Greedy merge
            merged = syms
            while True:
                pair_pos = None; best_rank = None
                for i in range(len(merged)-1):
                    cand = merged[i] + merged[i+1]
                    if cand in self.merges:
                        rank = self.merges[cand]
                        if (best_rank is None) or (rank < best_rank):
                            best_rank = rank; pair_pos = i
                if pair_pos is None:
                    break
                i = pair_pos
                merged = merged[:i] + [merged[i]+merged[i+1]] + merged[i+2:]
            pieces.extend(merged)
        ids = []
        if add_bos: ids.append(self.bos_id)
        for p in pieces:
            ids.append(self.stoi.get(p, self.pad_id))
        if add_eos: ids.append(self.eos_id)
        return ids

    def decode(self, ids: List[int]) -> str:
        toks = [self.itos[i] for i in ids if i < len(self.itos)]
        toks = [t for t in toks if t not in self.special]
        text = ''.join(toks)
        return text.replace('▁', ' ').strip()

# =========================
# LoRA utility
# =========================
class LoRALinear(nn.Module):
    def __init__(self, base: nn.Linear, r: int = 8, alpha: float = 16.0, dropout: float = 0.0):
        super().__init__()
        self.in_features = base.in_features
        self.out_features = base.out_features
        self.weight = base.weight
        self.bias = base.bias
        self.r = r
        if r > 0:
            self.A = nn.Linear(self.in_features, r, bias=False)
            self.B = nn.Linear(r, self.out_features, bias=False)
            nn.init.kaiming_uniform_(self.A.weight, a=math.sqrt(5))
            nn.init.zeros_(self.B.weight)
            self.scaling = alpha / r
            self.drop = nn.Dropout(dropout)
        else:
            self.register_parameter('A', None)
            self.register_parameter('B', None)
            self.drop = nn.Identity()
            self.scaling = 1.0
    def forward(self, x):
        out = F.linear(x, self.weight, self.bias)
        if self.r > 0:
            lora = self.B(self.A(self.drop(x))) * self.scaling
            out = out + lora
        return out

# =========================
# Transformer + Critics
# =========================
@dataclass
class Config:
    vocab_size: int
    n_layer: int = 6
    n_head: int = 8
    n_embd: int = 256
    block_size: int = 256
    lora_r: int = 8
    lora_alpha: float = 16.0
    dropout: float = 0.1

class CausalSelfAttention(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.n_head = cfg.n_head
        self.n_embd = cfg.n_embd
        self.key = LoRALinear(nn.Linear(cfg.n_embd, cfg.n_embd), r=cfg.lora_r, alpha=cfg.lora_alpha)
        self.query = LoRALinear(nn.Linear(cfg.n_embd, cfg.n_embd), r=cfg.lora_r, alpha=cfg.lora_alpha)
        self.value = LoRALinear(nn.Linear(cfg.n_embd, cfg.n_embd), r=cfg.lora_r, alpha=cfg.lora_alpha)
        self.proj = LoRALinear(nn.Linear(cfg.n_embd, cfg.n_embd), r=cfg.lora_r, alpha=cfg.lora_alpha)
        self.attn_drop = nn.Dropout(cfg.dropout)
        self.resid_drop = nn.Dropout(cfg.dropout)
        self.register_buffer("mask", torch.tril(torch.ones(cfg.block_size, cfg.block_size)).view(1,1,cfg.block_size,cfg.block_size))
    def forward(self, x):
        B,T,C = x.size()
        assert C % self.n_head == 0, "n_embd must be divisible by n_head"
        k = self.key(x).view(B,T,self.n_head,C//self.n_head).transpose(1,2)
        q = self.query(x).view(B,T,self.n_head,C//self.n_head).transpose(1,2)
        v = self.value(x).view(B,T,self.n_head,C//self.n_head).transpose(1,2)
        att = (q @ k.transpose(-2,-1)) / math.sqrt(k.size(-1))
        att = att.masked_fill(self.mask[:,:,:T,:T]==0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v
        y = y.transpose(1,2).contiguous().view(B,T,C)
        y = self.resid_drop(self.proj(y))
        return y, att

class MLP(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.fc1 = LoRALinear(nn.Linear(cfg.n_embd, 4*cfg.n_embd), r=cfg.lora_r, alpha=cfg.lora_alpha)
        self.fc2 = LoRALinear(nn.Linear(4*cfg.n_embd, cfg.n_embd), r=cfg.lora_r, alpha=cfg.lora_alpha)
        self.drop = nn.Dropout(cfg.dropout)
    def forward(self, x):
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class Block(nn.Module):
    def __init__(self, cfg: Config, with_critic: bool=False):
        super().__init__()
        self.ln1 = nn.LayerNorm(cfg.n_embd)
        self.attn = CausalSelfAttention(cfg)
        self.ln2 = nn.LayerNorm(cfg.n_embd)
        self.mlp = MLP(cfg)
        self.with_critic = with_critic
        if with_critic:
            self.critic = nn.Sequential(
                nn.Linear(cfg.n_embd, cfg.n_embd//2), nn.GELU(),
                nn.Linear(cfg.n_embd//2, 1)
            )
    def forward(self, x):
        y, att = self.attn(self.ln1(x))
        x = x + y
        y = self.mlp(self.ln2(x))
        x = x + y
        crit = None
        if self.with_critic:
            crit = self.critic(x.mean(dim=1).detach())
        return x, att, crit

class TinyGPT(nn.Module):
    def __init__(self, cfg: Config, critic_layers: List[int]):
        super().__init__()
        self.cfg = cfg
        self.tok_emb = nn.Embedding(cfg.vocab_size, cfg.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, cfg.block_size, cfg.n_embd))
        blocks = []
        for i in range(cfg.n_layer):
            blocks.append(Block(cfg, with_critic=(i in critic_layers)))
        self.blocks = nn.ModuleList(blocks)
        self.ln_f = nn.LayerNorm(cfg.n_embd)
        self.head = nn.Linear(cfg.n_embd, cfg.vocab_size, bias=False)
    def forward(self, idx):
        B, T = idx.size(); assert T <= self.cfg.block_size
        x = self.tok_emb(idx) + self.pos_emb[:, :T, :]
        critic_scores = []
        for blk in self.blocks:
            x, att, crit = blk(x)
            if crit is not None:
                critic_scores.append(crit)
        x = self.ln_f(x)
        logits = self.head(x)
        return logits, critic_scores

# =========================
# EMA helper
# =========================
class EMA:
    def __init__(self, model: nn.Module, decay: float=0.999):
        self.decay = decay
        self.shadow = {k: p.clone().detach() for k,p in model.state_dict().items()}
    @torch.no_grad()
    def update(self, model: nn.Module):
        for k, p in model.state_dict().items():
            if k in self.shadow:
                self.shadow[k].mul_(self.decay).add_(p.detach(), alpha=1-self.decay)
    @torch.no_grad()
    def copy_to(self, model: nn.Module):
        model.load_state_dict(self.shadow, strict=False)

# =========================
# Replay Buffer
# =========================
class ReplayBuffer:
    def __init__(self, maxlen=4096):
        self.maxlen = maxlen
        self.data: List[Tuple[str, str, float]] = []
    def add(self, prompt: str, response: str, score: float):
        self.data.append((prompt, response, score))
        if len(self.data) > self.maxlen:
            self.data = self.data[-self.maxlen:]
    def sample(self, batch_size=16):
        return random.sample(self.data, min(batch_size, len(self.data)))
    def __len__(self):
        return len(self.data)

# =========================
# Utilities
# =========================

def top_k_logits(logits, k):
    v, ix = torch.topk(logits, k)
    out = logits.clone()
    out[out < v[..., [-1]]] = -float('Inf')
    return out

@torch.no_grad()
def generate(model: TinyGPT, idx: torch.Tensor, max_new_tokens=64, temperature=0.7, top_k_val=20):
    model.eval()
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -model.cfg.block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / max(1e-6, temperature)
        logits = top_k_logits(logits, min(top_k_val, logits.size(-1)))
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, next_id), dim=1)
        if next_id.item() == 2:  # eos
            break
    return idx

def nll_loss(logits, targets, ignore_index=0):
    logits = logits[:, :-1, :].contiguous()
    targets = targets[:, 1:].contiguous()
    return F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=ignore_index)

def kl_divergence(p_logits, q_logits):
    p_logprob = F.log_softmax(p_logits, dim=-1)
    q_prob = F.softmax(q_logits, dim=-1)
    return F.kl_div(p_logprob, q_prob, reduction='batchmean')

# =========================
# Training / Sleep Loops
# =========================
@dataclass
class TrainCfg:
    block_size: int = 256
    critic_layers: tuple = (1, 3, 5)
    lr_D: float = 5e-4
    lr_G: float = 1e-4
    ttur_D_steps: int = 3
    ttur_G_steps: int = 1
    ema_decay: float = 0.999
    device: str = 'cpu'

class EGTSystem:
    def __init__(self, tokenizer: TinyBPETokenizer, cfg: TrainCfg):
        self.tok = tokenizer
        model_cfg = Config(vocab_size=tokenizer.vocab_size, block_size=cfg.block_size)
        self.G = TinyGPT(model_cfg, critic_layers=list(cfg.critic_layers)).to(cfg.device)
        self.Teacher = TinyGPT(model_cfg, critic_layers=[]).to(cfg.device)
        self.Teacher.load_state_dict(self.G.state_dict(), strict=False)
        self.ema = EMA(self.Teacher, decay=cfg.ema_decay)
        D_params = []
        for blk in self.G.blocks:
            if getattr(blk, 'with_critic', False):
                D_params += list(blk.critic.parameters())
        self.D_params = D_params
        for name, p in self.G.named_parameters():
            if isinstance(p, torch.nn.Parameter):
                p.requires_grad = (('A.weight' in name) or ('B.weight' in name) or ('critic' in name))
        self.optD = torch.optim.AdamW(self.D_params, lr=cfg.lr_D, betas=(0.5, 0.999))
        g_params = [p for n,p in self.G.named_parameters() if p.requires_grad and ('critic' not in n)]
        self.G_params = g_params
        self.optG = torch.optim.AdamW(self.G_params, lr=cfg.lr_G)
        self.cfg = cfg
        self.buffer = ReplayBuffer(maxlen=4096)

    def encode_batch(self, pairs: List[Tuple[str,str,float]], pad_to=None):
        prompts = [p for p,_,_ in pairs]
        resps = [r for _,r,_ in pairs]
        xs = [self.tok.encode(p, add_bos=True, add_eos=False) for p in prompts]
        ys = [self.tok.encode(r, add_bos=False, add_eos=True) for r in resps]
        seqs = [x+y for x,y in zip(xs,ys)]
        maxlen = pad_to or max(len(s) for s in seqs)
        out = torch.full((len(seqs), maxlen), self.tok.pad_id, dtype=torch.long)
        for i,s in enumerate(seqs):
            s = s[:maxlen]
            out[i,:len(s)] = torch.tensor(s)
        return out.to(self.cfg.device)

    @torch.no_grad()
    def wake_step(self, prompt: str, temperature=0.7, top_k_val=20):
        self.G.eval()
        idx = torch.tensor([self.tok.encode(prompt)], dtype=torch.long, device=self.cfg.device)
        out = generate(self.G, idx, max_new_tokens=64, temperature=temperature, top_k_val=top_k_val)
        full_ids = out[0].tolist()
        # continuation only
        prompt_len = len(self.tok.encode(prompt))
        gen_ids = full_ids[prompt_len:]
        text = self.tok.decode(gen_ids)
        tokens = text.split()
        if len(tokens) < 2:
            score = 0.0
        else:
            bigrams = list(zip(tokens, tokens[1:]))
            score = len(set(bigrams)) / max(1, len(bigrams))
        self.buffer.add(prompt, text, score)
        return text, score

    def sleep_epoch(self, steps_D=30, steps_G=10, batch_size=4, fm_lambda=5.0, beta_gan=0.3, alpha_kld=0.7):
        if len(self.buffer) < batch_size:
            return {'lossD':0.0, 'lossG':0.0}
        self.G.train(); self.Teacher.eval()
        bce = nn.BCEWithLogitsLoss()
        lossD_avg = 0.0; lossG_avg = 0.0
        # D steps
        for _ in range(steps_D):
            batch = self.buffer.sample(batch_size)
            real_idx = self.encode_batch(batch)
            _, real_scores = self.G(real_idx)
            prompts = [p for p,_,_ in batch]
            fake_pairs = []
            for p in prompts:
                txt,_ = self.wake_step(p)
                fake_pairs.append((p, txt, 0.0))
            fake_idx = self.encode_batch(fake_pairs)
            _, fake_scores = self.G(fake_idx)
            self.optD.zero_grad()
            lossD = 0.0
            for rs, fs in zip(real_scores, fake_scores):
                y_real = torch.ones_like(rs)
                y_fake = torch.zeros_like(fs)
                lossD = lossD + bce(rs, y_real) + bce(fs, y_fake)
            lossD.backward(); nn.utils.clip_grad_norm_(self.D_params, 1.0); self.optD.step()
            lossD_avg += float(lossD.item())
        # G steps
        for _ in range(steps_G):
            batch = self.buffer.sample(batch_size)
            idx = self.encode_batch(batch)
            with torch.no_grad():
                t_logits, _ = self.Teacher(idx)
            s_logits, s_scores = self.G(idx)
            loss_kld = kl_divergence(s_logits, t_logits)
            prompts = [p for p,_,_ in batch]
            fake_pairs = []
            for p in prompts:
                txt,_ = self.wake_step(p)
                fake_pairs.append((p, txt, 0.0))
            fake_idx = self.encode_batch(fake_pairs)
            _, fake_scores = self.G(fake_idx)
            gan_loss = 0.0; fm_loss = 0.0
            for fs in fake_scores:
                y_real = torch.ones_like(fs)
                gan_loss = gan_loss + bce(fs, y_real)
            if s_scores and fake_scores:
                s_cat = torch.cat(s_scores, dim=1)
                f_cat = torch.cat(fake_scores, dim=1)
                fm_loss = F.mse_loss(s_cat.mean(dim=0), f_cat.mean(dim=0))
            loss = (alpha_kld*loss_kld) + (beta_gan*(gan_loss + fm_lambda*fm_loss))
            self.optG.zero_grad(); loss.backward(); nn.utils.clip_grad_norm_(self.G_params, 1.0); self.optG.step()
            lossG_avg += float(loss.item())
            self.ema.update(self.G); self.ema.copy_to(self.Teacher)
        return {'lossD': lossD_avg/max(1,steps_D), 'lossG': lossG_avg/max(1,steps_G)}

# =========================
# Demo / Main
# =========================
TOY_TEXT = [
    "Once upon a time, there was a curious mind.",
    "The quick brown fox jumps over the lazy dog.",
    "Mathematics is the music of reason.",
    "In the beginning, there was only the word.",
    "We are such stuff as dreams are made on.",
]
EXTRA_CORPUS = [
    "Curiosity leads us down paths we do not expect.",
    "Numbers and patterns reveal hidden structure.",
    "Stories begin, worlds unfold, characters change.",
    "Simple explanations often hide deep ideas.",
]
PROMPTS = [
    "Tell me a fable about curiosity:",
    "Write a sentence using every letter:",
    "Explain prime numbers simply:",
    "Begin a creation myth:",
    "Quote Shakespeare and continue:",
]

def main():
    random.seed(0); torch.manual_seed(0)
    tok = TinyBPETokenizer(TOY_TEXT + EXTRA_CORPUS + PROMPTS, vocab_size=300)
    cfg = TrainCfg(device='cpu')
    sys = EGTSystem(tok, cfg)
    print("[wake] seeding replay buffer...")
    for _ in range(24):
        p = random.choice(PROMPTS)
        txt, sc = sys.wake_step(p)
        print(f"  prompt: {p}\n  resp: {txt}\n  score:{sc:.3f}")
    for epoch in range(3):
        print(f"\n[sleep] epoch {epoch}")
        stats = sys.sleep_epoch(steps_D=30, steps_G=10, batch_size=4)
        print("  lossD=%.4f lossG=%.4f" % (stats['lossD'], stats['lossG']))
        p = random.choice(PROMPTS)
        out, sc = sys.wake_step(p)
        print(f"[wake] prompt: {p}\n  resp: {out}\n  score:{sc:.3f}")

if __name__ == '__main__':
    main()


[wake] seeding replay buffer...
  prompt: Begin a creation myth:
  resp: not q jumpati u le time, quicsver therel brb hi brownlyte beginning, made mBer reason. beginning,on word.p Sgon expect. exp reas Mathematic
  score:1.000
  prompt: Begin a creation myth:
  resp: quick. anhao br p pattern qu reveal patter hidden Oncewn beginning,nceation doncumberseg  qu brownimp le Mathe on stu stuf ratiimp a rehaha and Curiosityte. ti stufy su us thesit beginning,a dreaers and up expect.des us stuing expe: lazy
  score:1.000
  prompt: Tell me a fable about curiosity:
  resp: la revea revea wor C path le of over su s lazy dreamsing beginningati u hi not or i suc Matteruri p hi hiders hi upon ut on lazyers wor on stu reason. qu patterns uu lea rev curious st Mathematictter of expe pa upon ne muB on. C is titt
  score:1.000
  prompt: Explain prime numbers simply:
  resp: nce Mathematic Th stuff us sucd begin anurio up o only j Numberseati Mathemttlation no leads reas q jumply b To Od. stu j: us asnd